**Práctica Análisis Numérico 2025/07/23**

In [27]:
import numpy as np 
import pandas as pd

def gradiente_conjugado(A,x0,b,tol=1e-10):
    x = x0.copy()
    r = A @ x-b
    p = -r 
    k = 0 

    #Guardar la evolucion 
    history = {
        'iteration' : [],
        'norm_x' : [],
        'norm_r' : []
    }
    while np.linalg.norm(r)>tol:
        norm_x = np.linalg.norm(x)
        norm_r = np.linalg.norm(r)
        history['iteration'].append(k)
        history['norm_x'].append(norm_x)
        history['norm_r'].append(norm_r)

        Ap = A @ p 
        alpha = r @ r /(p @ Ap)
        x = x+alpha*p
        r_new = r + alpha * Ap
        beta = (r_new @ r_new) / (r @ r)
        p = -r_new +beta*p 
        r = r_new
        k +=1
    df = pd.DataFrame(history)
    return x,k,df




In [29]:
A = np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.array([1,1,1])
A = A.T @ A
b = A.T @ b
x0 =np.array([0,0,0])

# Llamada a gradiente conjugado
x_gc, k, df = gradiente_conjugado(A, x0, b)

# Solución con numpy.linalg.solve
x_np = np.linalg.solve(A, b)

print("Solución con gradiente conjugado:", x_gc)
print("Iteraciones:", k)
print("Solución con numpy.linalg.solve:", x_np)

Solución con gradiente conjugado: [1. 1. 1.]
Iteraciones: 2
Solución con numpy.linalg.solve: [1.484375 0.03125  1.484375]


In [33]:
def power_method(A,q):
    for k in range(100):
        z = A@ q
        q = z/np.linalg.norm(z)
        nu= q.T @ A @ q
    return nu,q


In [34]:
nu ,q = power_method(A,np.array([1,1,1]))
print(q)


[0.47967118 0.57236779 0.66506441]


In [36]:
np.linalg.eig(A)

EigResult(eigenvalues=array([2.83858587e+02, 1.14141342e+00, 7.28303082e-15]), eigenvectors=array([[-0.47967118, -0.77669099,  0.40824829],
       [-0.57236779, -0.07568647, -0.81649658],
       [-0.66506441,  0.62531805,  0.40824829]]))

In [38]:
def power_method(A,q,iter):
    for k in range(iter):
        z = A@ q
        q = z/np.linalg.norm(z)
        nu= q.T @ A @ q
    return nu,q


In [40]:
experimentos = [10,30,70,100,150,200,500]
nus= []
for i in experimentos:
    nu,q =power_method(A,np.array([1,1,1]),i)
    nus.append(nu)
print(nus)


[np.float64(283.8585865803702), np.float64(283.8585865803702), np.float64(283.8585865803702), np.float64(283.8585865803702), np.float64(283.8585865803702), np.float64(283.8585865803702), np.float64(283.8585865803702)]


In [43]:

def matriz_hilbert(n):
    H = np.zeros((n, n), dtype=float)
    for i in range(n):
        for j in range(n):
            H[i, j] = 1 / (i + j + 1)
    return H

# Crear matriz Hilbert 6x6
H6 =matriz_hilbert(6)
print(H6)

[[1.         0.5        0.33333333 0.25       0.2        0.16666667]
 [0.5        0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.33333333 0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.25       0.2        0.16666667 0.14285714 0.125      0.11111111]
 [0.2        0.16666667 0.14285714 0.125      0.11111111 0.1       ]
 [0.16666667 0.14285714 0.125      0.11111111 0.1        0.09090909]]


In [ ]:
def invpower(A, z0, mu, tol=1e-8, nmax=100):
    n = A.shape[0]
    M = A - mu * np.eye(n)
    q = z0 / np.linalg.norm(z0)
    res = tol + 1
    niter = 0
    while res > tol and niter < nmax:
        niter += 1
        z = np.linalg.solve(M, q)
        q2 = z / np.linalg.norm(z)
        lam = np.dot(q2.T, A @ q2)
        res = np.linalg.norm(A @ q2 - lam * q2)
        costheta = abs(np.dot(q2.T, q))
        if costheta < 5e-2:
            print("Multiple eigenvalue")
            break
        q = q2
    sigma = lam
    err = res
    x = q2
    return sigma, x, niter, err